### Sampling from an mps

In [22]:
from mps.utils import build_uniform_product_state
import numpy as np
import torch

In [6]:
theta, phi = np.pi/2, 0
L=4

This just creates a product state of spins aligned in the X direction

In [7]:
psi = build_uniform_product_state(L,theta, phi)

In [10]:
print(psi.shape, psi.norm())

[(1, 1), (1, 1), (1, 1), (1, 1)] tensor(1.0000, grad_fn=<SqueezeBackward0>)


#### Sample from the mps in the z (ie standard) basis

In [18]:
# how many measurements to take
Nsamp = 10

In [19]:
samples = psi.sample(Nsamp)

Have a look at the sampled data:

In [20]:
print(samples)

tensor([[0, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 0],
        [1, 1, 1, 1],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1]])


It's super high entropy, because each spin is in an equal superposition in the z basis. Now try sampling in the X basis, ie after applying a pi/2 rotation:

In [24]:
#helper function to compute exponentials of pauli matrices
from mps.qtools import pauli_exp
#it expects torch tensors, which specify rotation angles for each sample, at each site
theta_per_sample = theta * torch.ones(Nsamp,L)
phi_per_sample = phi * torch.ones(Nsamp,L)
# a ComplexTensor which defines the rotation at each site, for each sample (here they're all the same...)
rotations = pauli_exp(theta_per_sample, phi_per_sample)
#shape = (num samples, system size, 2, 2)
print(rotations.shape)

torch.Size([10, 4, 2, 2])


In [25]:
samples_x = psi.sample(Nsamp, rotations=rotations)

Now that we're measuring in eigenbasis, no entropy:

In [27]:
print(samples_x)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])


Try an entangled state -- a phase-free GHZ state:

In [29]:
from mps.utils import build_ghz_plus

In [30]:
psi = build_ghz_plus(L)
samples = psi.sample(Nsamp)

In [31]:
print(samples)

tensor([[1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 1, 0, 1],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 1, 0, 1],
        [0, 1, 0, 1],
        [0, 1, 0, 1],
        [0, 1, 0, 1],
        [1, 0, 1, 0]])
